# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5a9a8afaf0>
├── 'a' --> tensor([[-0.6694, -0.8235, -0.0858],
│                   [ 0.6345,  0.8203,  0.1235]])
└── 'x' --> <FastTreeValue 0x7f5a9a8afa30>
    └── 'c' --> tensor([[-1.2600, -1.0089, -0.3291, -0.0543],
                        [-0.7754, -0.9687, -2.4649,  0.4630],
                        [ 0.9184,  0.7694, -1.2430, -1.7048]])

In [4]:
t.a

tensor([[-0.6694, -0.8235, -0.0858],
        [ 0.6345,  0.8203,  0.1235]])

In [5]:
%timeit t.a

75.2 ns ± 0.537 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5a9a8afaf0>
├── 'a' --> tensor([[-0.1137,  2.4439,  0.0168],
│                   [ 0.7883,  0.5226, -0.0136]])
└── 'x' --> <FastTreeValue 0x7f5a9a8afa30>
    └── 'c' --> tensor([[-1.2600, -1.0089, -0.3291, -0.0543],
                        [-0.7754, -0.9687, -2.4649,  0.4630],
                        [ 0.9184,  0.7694, -1.2430, -1.7048]])

In [7]:
%timeit t.a = new_value

90 ns ± 0.548 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6694, -0.8235, -0.0858],
               [ 0.6345,  0.8203,  0.1235]]),
    x: Batch(
           c: tensor([[-1.2600, -1.0089, -0.3291, -0.0543],
                      [-0.7754, -0.9687, -2.4649,  0.4630],
                      [ 0.9184,  0.7694, -1.2430, -1.7048]]),
       ),
)

In [10]:
b.a

tensor([[-0.6694, -0.8235, -0.0858],
        [ 0.6345,  0.8203,  0.1235]])

In [11]:
%timeit b.a

72.1 ns ± 2.99 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7292, -0.4783, -0.2165],
               [ 1.2986, -0.1175,  0.2131]]),
    x: Batch(
           c: tensor([[-1.2600, -1.0089, -0.3291, -0.0543],
                      [-0.7754, -0.9687, -2.4649,  0.4630],
                      [ 0.9184,  0.7694, -1.2430, -1.7048]]),
       ),
)

In [13]:
%timeit b.a = new_value

609 ns ± 30.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

951 ns ± 4.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12 µs ± 62.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

165 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

163 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5a9a8af2b0>
├── 'a' --> tensor([[[-0.6694, -0.8235, -0.0858],
│                    [ 0.6345,  0.8203,  0.1235]],
│           
│                   [[-0.6694, -0.8235, -0.0858],
│                    [ 0.6345,  0.8203,  0.1235]],
│           
│                   [[-0.6694, -0.8235, -0.0858],
│                    [ 0.6345,  0.8203,  0.1235]],
│           
│                   [[-0.6694, -0.8235, -0.0858],
│                    [ 0.6345,  0.8203,  0.1235]],
│           
│                   [[-0.6694, -0.8235, -0.0858],
│                    [ 0.6345,  0.8203,  0.1235]],
│           
│                   [[-0.6694, -0.8235, -0.0858],
│                    [ 0.6345,  0.8203,  0.1235]],
│           
│                   [[-0.6694, -0.8235, -0.0858],
│                    [ 0.6345,  0.8203,  0.1235]],
│           
│                   [[-0.6694, -0.8235, -0.0858],
│                    [ 0.6345,  0.8203,  0.1235]]])
└── 'x' --> <FastTreeValue 0x7f5a9a8af670>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37 µs ± 342 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5a988286a0>
├── 'a' --> tensor([[-0.6694, -0.8235, -0.0858],
│                   [ 0.6345,  0.8203,  0.1235],
│                   [-0.6694, -0.8235, -0.0858],
│                   [ 0.6345,  0.8203,  0.1235],
│                   [-0.6694, -0.8235, -0.0858],
│                   [ 0.6345,  0.8203,  0.1235],
│                   [-0.6694, -0.8235, -0.0858],
│                   [ 0.6345,  0.8203,  0.1235],
│                   [-0.6694, -0.8235, -0.0858],
│                   [ 0.6345,  0.8203,  0.1235],
│                   [-0.6694, -0.8235, -0.0858],
│                   [ 0.6345,  0.8203,  0.1235],
│                   [-0.6694, -0.8235, -0.0858],
│                   [ 0.6345,  0.8203,  0.1235],
│                   [-0.6694, -0.8235, -0.0858],
│                   [ 0.6345,  0.8203,  0.1235]])
└── 'x' --> <FastTreeValue 0x7f5a6ccc2c70>
    └── 'c' --> tensor([[-1.2600, -1.0089, -0.3291, -0.0543],
                        [-0.7754, -0.9687, -2.4649,  0.4630],
                 

In [23]:
%timeit t_cat(trees)

34.3 µs ± 248 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

70.6 µs ± 1.95 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.2600, -1.0089, -0.3291, -0.0543],
                       [-0.7754, -0.9687, -2.4649,  0.4630],
                       [ 0.9184,  0.7694, -1.2430, -1.7048]],
              
                      [[-1.2600, -1.0089, -0.3291, -0.0543],
                       [-0.7754, -0.9687, -2.4649,  0.4630],
                       [ 0.9184,  0.7694, -1.2430, -1.7048]],
              
                      [[-1.2600, -1.0089, -0.3291, -0.0543],
                       [-0.7754, -0.9687, -2.4649,  0.4630],
                       [ 0.9184,  0.7694, -1.2430, -1.7048]],
              
                      [[-1.2600, -1.0089, -0.3291, -0.0543],
                       [-0.7754, -0.9687, -2.4649,  0.4630],
                       [ 0.9184,  0.7694, -1.2430, -1.7048]],
              
                      [[-1.2600, -1.0089, -0.3291, -0.0543],
                       [-0.7754, -0.9687, -2.4649,  0.4630],
                       [ 0.9184,  0.7694, -1.2430, -1.7048]],

In [26]:
%timeit Batch.stack(batches)

112 µs ± 730 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.2600, -1.0089, -0.3291, -0.0543],
                      [-0.7754, -0.9687, -2.4649,  0.4630],
                      [ 0.9184,  0.7694, -1.2430, -1.7048],
                      [-1.2600, -1.0089, -0.3291, -0.0543],
                      [-0.7754, -0.9687, -2.4649,  0.4630],
                      [ 0.9184,  0.7694, -1.2430, -1.7048],
                      [-1.2600, -1.0089, -0.3291, -0.0543],
                      [-0.7754, -0.9687, -2.4649,  0.4630],
                      [ 0.9184,  0.7694, -1.2430, -1.7048],
                      [-1.2600, -1.0089, -0.3291, -0.0543],
                      [-0.7754, -0.9687, -2.4649,  0.4630],
                      [ 0.9184,  0.7694, -1.2430, -1.7048],
                      [-1.2600, -1.0089, -0.3291, -0.0543],
                      [-0.7754, -0.9687, -2.4649,  0.4630],
                      [ 0.9184,  0.7694, -1.2430, -1.7048],
                      [-1.2600, -1.0089, -0.3291, -0.0543],
                   

In [28]:
%timeit Batch.cat(batches)

205 µs ± 2.99 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

383 µs ± 4.45 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
